## camera calibration
-
-
-


### obtain the camera matrix

In [1]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:7].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,7),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,7), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(2000)
        
cv2.destroyAllWindows()


ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)




### print the camera matrix

In [2]:
# print the camera matrix
print(mtx)


[[ 845.85589796    0.          291.79992742]
 [   0.          796.58787089  278.80038013]
 [   0.            0.            1.        ]]


In [3]:
import numpy
from numpy import array

mtx_list = array(mtx).flatten().tolist()


In [4]:
print(mtx_list)

[845.8558979590817, 0.0, 291.79992741670543, 0.0, 796.5878708869893, 278.8003801342742, 0.0, 0.0, 1.0]


## generate yaml document 

In [5]:

with open('test_data.yaml', 'w') as f:


    image_width = 640
    image_height = 480

    #*****************************************   camera matrix   ***********************************************


    camera_name = '\"0800461000822014\"' 
    # camera_matrix 
    rows = 3
    cols = 3
    data = mtx_list


    image_width = 'image_width: {}\n'.format(image_width)
    image_height ='image_height: {}\n'. format(image_height)
    camera_name ='camera_name: {}\n'. format(camera_name)
    # camera_matrix :
    rows = '  rows: {}\n'. format(rows)
    cols = '  cols: {}\n'.format(cols)
    data = '  data: {}\n'.format(data)


    f.write(image_width)
    f.write(image_height )
    f.write(camera_name)
    f.write('camera_matrix: \n')
    f.write(rows)
    f.write(cols)
    f.write(data)


    #***************************************  distortion matrix   **********************************************

    distortion_model = 'plumb_bob'
    # distortion_coefficients:

    rows = 1
    cols = 5
    data = [-0.225495, -0.363048, -0.000477994, -0.000132753, 0]

    distortion_model = 'distortion_model: {}\n'.format(distortion_model)
    rows = '  rows: {}\n'. format(rows)
    cols = '  cols: {}\n'.format(cols)
    data = '  data: {}\n'.format(data)

    f.write(distortion_model)
    f.write('distortion_coefficients: \n')
    f.write(rows)
    f.write(cols)
    f.write(data)


    #**************************************  rectification_matrix  *********************************************

    rows = 3
    cols = 3
    data = [1, 0, 0, 0, 1, 0, 0, 0, 1]

    rows = '  rows: {}\n'. format(rows)
    cols = '  cols: {}\n'.format(cols)
    data = '  data: {}\n'.format(data)

    f.write('rectification_matrix: \n')
    f.write(rows)
    f.write(cols)
    f.write(data)



    #****************************************  projection_matrix  ********************************************** 

    rows = 3
    cols = 4
    data = [2302.59, 0, 610.756, 0, 0, 2309.46, 506.539, 0, 0, 0, 1, 0]

    rows = '  rows: {}\n'. format(rows)
    cols = '  cols: {}\n'.format(cols)
    data = '  data: {}\n'.format(data)

    f.write('projection_matrix: \n')
    f.write(rows)
    f.write(cols)
    f.write(data)


### generate yaml document

issue:
can't output a list in one row

https://github.com/Lizmango/CV/issues/7

In [ ]:
import yaml


mat = dict(
    image_width = 1280,
    image_height = 960,
    camera_name = "xiaohua",
    camera_matrix = dict(
        rows = 3,
        cols = 3,
        data = mtx)
)
with open('mat_data.yaml', 'w') as outfile:
    yaml.dump(mat, outfile, default_flow_style=False)
print(mtx)

### define the matx_get function to get matrix from yaml document 

**warning:**

  wrong format of output
  
**code**
```py
    print('camera_matrix \n',cam)
    print('distortion_coefficients \n',dis)
    print('projection_matrix \n',pro)
    print('rectification_matrix \n',rec)
```
**output**
```py
('camera_matrix \n', matrix([[  2.30259000e+03,   0.00000000e+00,   6.10756000e+02],
        [  0.00000000e+00,   2.30946000e+03,   5.06539000e+02],
        [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]]))
('distortion_coefficients \n', matrix([[ -2.25495000e-01,  -3.63048000e-01,  -4.77994000e-04,
          -1.32753000e-04,   0.00000000e+00]]))
('projection_matrix \n', matrix([[  2.30259000e+03,   0.00000000e+00,   6.10756000e+02,
           0.00000000e+00],
        [  0.00000000e+00,   2.30946000e+03,   5.06539000e+02,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
           0.00000000e+00]]))
('rectification_matrix \n', matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]))
```


In [6]:
file_name = ('camera_matrix.yaml')

In [7]:
import yaml
import numpy as np

def matx_get(file_name):
    f = open(file_name)
    mtrx = yaml.load(f)
    
    cam = np.matrix(mtrx['camera_matrix']['data']).reshape(3,3)
    dis = np.matrix(mtrx['distortion_coefficients']['data']).reshape(1,5) 
    pro = np.matrix(mtrx['projection_matrix']['data']).reshape(3,4)
    rec = np.matrix(mtrx['rectification_matrix']['data']).reshape(3,3)
    
    print("camera_matrix:  \n {}".format(cam))
    print("distortion_coefficients:  \n {}".format(dis))
    print("projection_matrix:  \n {}".format(pro))
    print("rectification_matrix:  \n {}".format(rec))

matx_get('camera_matrix.yaml')   

camera_matrix:  
 [[  2.30259000e+03   0.00000000e+00   6.10756000e+02]
 [  0.00000000e+00   2.30946000e+03   5.06539000e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
distortion_coefficients:  
 [[ -2.25495000e-01  -3.63048000e-01  -4.77994000e-04  -1.32753000e-04
    0.00000000e+00]]
projection_matrix:  
 [[  2.30259000e+03   0.00000000e+00   6.10756000e+02   0.00000000e+00]
 [  0.00000000e+00   2.30946000e+03   5.06539000e+02   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00]]
rectification_matrix:  
 [[1 0 0]
 [0 1 0]
 [0 0 1]]


### using undistort

In [8]:
#using undistort
img = cv2.imread('chess08.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult1.png',dst)

True

### using remaping

In [9]:
#using remaping
# undistort
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult2.png',dst)

True

### calculate error

In [10]:
mean_error = 0
tot_error =0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error
    print("tot_error %f"%(tot_error))

    
print ("total error: %f"%(tot_error/len(objpoints)))

tot_error 0.109478
tot_error 0.233759
tot_error 0.318128
tot_error 0.390566
tot_error 0.483587
tot_error 0.561234
tot_error 0.674374
tot_error 0.762033
tot_error 0.837495
tot_error 0.911084
tot_error 1.005607
tot_error 1.139577
tot_error 1.233083
tot_error 1.292589
total error: 0.092328
